In [16]:
# BIOL 51000 - Data Systems Life Sciences
# Week 5 Coding Assignment: Calculating RMSD and Aligning a Structure Ensemble
# Mathew Golf

In [1]:
cd C:/Users/Mathe/Desktop/PythonForBiology

C:\Users\Mathe\Desktop\PythonForBiology


In [2]:
# Import Libraries
from numpy import zeros, ones, sqrt, linalg, exp, identity, array, dot
ATOMIC_NUMS = {'H':1, 'C':12, 'N':14, 'O':16, 'P':31, 'S':32}

In [3]:
# Calculate variation in coordinates across the atom positions represented in arrays
# differences in coordinate positions, square them, find average and square the root
# average distance of coordinate spread -> average of squares not distances 
# = bias towards larger deviations having more influence

def calcRmsds (refCoords, allCoords, weights):
    """ takes array of reference coordinates, list of the other coordinate
    array for comparison and a list of weights to bias atoms separately """
    rmsds = [] # hold values
    totalWeight = sum(weights) # total of all input weights
    totalSquares = zeros(refCoords.shape) # empty array of same size to hold summation of positional differences
    
    # loop through coordinate arrays comparing to reference
    # whole array object manipulations - applied to all elements of the array
    # delta = array of all coordinate differences, squaring the elements
    for coords in allCoords:
        delta = coords - refCoords
        squares = delta * delta
        totalSquares += squares
        sumSquares = weights*squares.sum(axis=1) # squared deviation for each atom
        # sum of square values along the spatial axis. multiply by weights for sumSquares
        rmsds.append(sqrt(sum(sumSquares)/totalWeight))
    
    nStruct = len(allCoords) 
    atomRmsds = sqrt(totalSquares.sum(axis=1)/nStruct) # average of the RMSDs
    # for each atoms RMSD over the whole set of conformations
    
    return rmsds, atomRmsds
# finds optimum rotation to superimpose two arrays of coordinates

In [4]:
def alignCoords(coordsA, coordsB, weights=None):
    """ Takes two arrays of coordinates, optional array of weights, and finds
        the rotation which best superimposes pairs of coordinates """
    
    n = len(coordsA)
    if weights is None: # generates series of 1's same length as coordinates
        weights = ones(n)
        
    # find an optimum rotation transformation to minimize the differences in positions
    # between corresponding coordinates    
    rMat = dot(coordsB.transpose()*weights, coordsA) # weighted dot product of one 
    # coordinate array and the transpose of another = transformation of positions
    
    # Singular value decomposition (SVD) - factorises matrix into components
    # components are a rotation matrix, array of linear scaling factors, and an
    # opposing rotation matrix
    rMat1, sclaes, rMat2 = linalg.svd(rMat)
    
    # need to determine if mirror image so we multiply the detriments
    sign = linalg.det(rMat1) * linalg.det(rMat2)
    # detriment product determines our next action - if mirror image or not
    if sign < 0:
        rMat1[:, 2] *= -1 # flips the sign = remove reflection
    rotation = dot(rMat1, rMat2) # optimised rotation matrix
    coordsB = dot(coordsB, rotation) # apply coordinate transformation
    
    return rotation, coordsB

In [5]:
def getAtomCoords (structure):
    
    # initialize new numpy objects
    coords = []
    atoms = []
    
    # loop through all chains, residues, and atoms in the stucture adding to 
    # the initialized numpy objects. We know which coordinate for each atom
    # hierarchical molecular data -> numeric numpy array
    for chain in structure.chains:
        for residue in chain.residues:
            for atom in residue.atoms:
                coords.append(atom.coords)
                atoms.append(atom)
    
    return atoms, array(coords)

In [6]:
# weights paramter allows for different atoms to have different degrees of influence
# generally due to their relative atomic mass
def centerCoords(coords, weights):
   
    # transpose is done to ensure multiplication with three rows separately with weights
    wCoords = coords.transpose() * weights
    # sum along the rows [axis=1] -> array of three numbers
    xyzTotals = wCoords.sum(axis=1) 
    center = xyzTotals/sum(weights) # produces an average or center of coordinates
    coords -= center # move to new center by removing old center position
    
    return coords, center

In [7]:
# We want to superimpose more than two coordinate arrays -> superimpose all conformations
# of a whole structure ensemble via repeating pairwise superimpositions relative to reference.
# A better method would compare all against all at the same time - quantum programming?
def superimposeCoords(allCoords, weights, threshold=5.0):
    nStruct = len(allCoords)
    
    refCoords = allCoords[0] # set to first element of list of arrays
    meanCoords = zeros(refCoords.shape) # used to add coordinates after aligned to reference
    rotations = [] # starts empty and is passed back at the end once filled with alignments
    
    # first superimposition = loop all coordinate arrays aligning to reference
    for index, coords in enumerate(allCoords):
        if index == 0: # don't need to align with self as its the reference
            rotation = identity(3) # no rotation
        else: # coordinate alignment
        # optimised rotation matrix is updated and put back into allCoords
            rotation, coords = alignCoords(refCoords, coords, weights)
            allCoords[index] = coords # update to aligned - replaces previous
        
        rotations.append(rotation) # rotation matrix collected into rotations list
        meanCoords += coords # coordinate array added to total
    
    meanCoords /= nStruct # gives us the average positions
    rmsds, atomRmsds = calcRmsds(meanCoords, allCoords, weights) # calcs RMSDs and stores
    
    bestRmsd = min(rmsds) # best - or minimum value (smallest)
    bestIndex = rmsds.index(bestRmsd) # closest to mean - min index
    bestCoords = allCoords[bestIndex] # best set of coords now reference set for another round
    # bestCoords = reference array closest to mean of aligned coordinates
    
    # adjusted weights for second round scaled by threshold value
    # new weights have atoms with small RMSD values having largest weights
    # and as the variance increases the weighting reduces exponentially
    weightScale = atomRmsds/threshold
    weights *= exp(-weightScale*weightScale)
    
    meanCoords = bestCoords.copy() # average coordinates = closest to mean
    
    # skip indexes which match reference coords, and for all other indexes
    # superimposition alignment executed = optimising rotation to reference
    # we are inserted updated coords into allCoords at their index to then find
    # the new coord average after completion
    for index, coords in enumerate(allCoords):
        if index != bestIndex:
            rotation, coords = alignCoords(bestCoords, coords, weights)
            rotations[index] = rotation
            allCoords[index] = coords # updates to aligned
    
    meanCoords /= nStruct # new average coord set
    
    # compare new coords with average set of coords
    # new unbiased weights set to 1 via 'ones' = observed distance variation for each
    # atom is reported equally
    weights = ones(len(weights))
    rmsds, atomRmsds = calcRmsds(meanCoords, allCoords, weights)
    
    # optimized superimposition of coordinate arrays done
    return allCoords, rmsds, atomRmsds, rotations

In [8]:
# applies the optimised superimposition of coordinate arrays to structure objects
def superimposeStructures(structures):
    """ Takes a list of structure objects extracting coordinate arrays, 
        performs coordinate superimposition, and updates original Atom objects
        with new coordinates """
        
    weights = None # initialize
    allCoords = [] # all coordinates for all structures - list of 2D arrays
    
    # structure inspection - generating coordinate arrays and atom object lists
    for structure in structures:
        atoms, coords = getAtomCoords(structure)
        # defines weight list with atomic numbers - not very precise but fine
        if weights is None:
            weights = [ATOMIC_NUMS[atom.element] for atom in atoms]
            weights = array(weights)
        
        # redefine coordinates via moving to center
        coords, center = centerCoords(coords, weights)
        allCoords.append(coords) # centered array put into list containing all coords
        # run function to superimpose
        results = superimposeCoords(allCoords, weights)
        # Results = modified array coordinates, list of RMSD values for each 
        # structure and atom, list of rotations for the superimposition
        allCoords, rmsds, atomRmsds, rotations = results
    
    # generate list index and structure object from the structures
    for i, structure in enumerate(structures):
        atoms, oldCoords = getAtomCoords(structure) # list of atoms and array of original coords
        
        # update atom coords for each atom object to the new superimposed coords
        for j, atom in enumerate(atoms):
            atom.coords = allCoords[i][j]
            
    return rmsds, atomRmsds